In [137]:
train = pd.read_csv('train.csv',sep=';')

train['addnew'] = train.hours_since_last_cdm_error * train.avg_cdm_dispense_count_distinct_day 


In [138]:
train['addnew']

0       244.209335
1       223.403494
2       265.913216
3       301.880971
4       492.508980
5       472.872241
6       467.405941
7       492.396887
8       478.296316
9       544.767172
10      463.649241
11      459.239338
12      487.646485
13      556.667652
14      687.051091
15      379.212082
16      654.600561
17      751.006883
18      245.437500
19      182.363300
20      256.526946
21      348.286519
22      248.113111
23      242.675824
24      263.607073
25      242.912281
26      242.395452
27      235.602537
28      241.812019
29      246.371058
           ...    
6255    594.696095
6256    493.590231
6257    498.170347
6258    299.646908
6259    501.669236
6260    476.234465
6261    472.369759
6262    678.037593
6263    518.947139
6264    523.856361
6265    526.718995
6266    744.337733
6267    729.895833
6268    243.896471
6269    365.333637
6270    331.938325
6271    283.396569
6272    485.008696
6273    481.896385
6274    493.093813
6275    470.400000
6276    472.

In [134]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np
import pandas as pd
import lightgbm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import normalize

#
# Prepare the data
#

train = pd.read_csv('train.csv',sep=';')

train['addnew'] = train.Prices * train.Amount * orders_df.C

columns = ['cash_in_out','display_type','scanner_code_reader']
train.drop(columns, inplace=True, axis=1)

# get the labels
y = train.target.values
train.drop(['target'], inplace=True, axis=1)

x = normalize(train.values)
#
# Create training and validation sets
#
x, x_test, y, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)
#
# Create the LightGBM data containers
#

train_data = lightgbm.Dataset(x, label=y)
test_data = lightgbm.Dataset(x_test, label=y_test)
#
# Train the model
#

parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0
}

model = lightgbm.train(parameters,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=5000,
                       early_stopping_rounds=100)

print('Save model...')
# save model to file
# AUC 0.67 - 70 
model.save_model('model.txt')

[1]	valid_0's auc: 0.61663
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.639051
[3]	valid_0's auc: 0.653704
[4]	valid_0's auc: 0.656292
[5]	valid_0's auc: 0.658091
[6]	valid_0's auc: 0.659528
[7]	valid_0's auc: 0.655472
[8]	valid_0's auc: 0.655765
[9]	valid_0's auc: 0.659514
[10]	valid_0's auc: 0.659861
[11]	valid_0's auc: 0.662871
[12]	valid_0's auc: 0.661756
[13]	valid_0's auc: 0.661789
[14]	valid_0's auc: 0.663287
[15]	valid_0's auc: 0.664019
[16]	valid_0's auc: 0.665306
[17]	valid_0's auc: 0.663598
[18]	valid_0's auc: 0.663667
[19]	valid_0's auc: 0.664441
[20]	valid_0's auc: 0.666032
[21]	valid_0's auc: 0.666055
[22]	valid_0's auc: 0.667672
[23]	valid_0's auc: 0.670566
[24]	valid_0's auc: 0.672283
[25]	valid_0's auc: 0.671832
[26]	valid_0's auc: 0.673449
[27]	valid_0's auc: 0.672588
[28]	valid_0's auc: 0.672565
[29]	valid_0's auc: 0.673456
[30]	valid_0's auc: 0.674513
[31]	valid_0's auc: 0.675631
[32]	valid_0's auc: 0.673889
[33]	valid_0's auc:

In [127]:
#
# Create a submission
#
test = pd.read_csv('test.csv',sep=';')
columns = ['cash_in_out','display_type','scanner_code_reader']
test.drop(columns, inplace=True, axis=1)
x = test.values
#submission = pd.read_csv('out.csv')
ids = test['atm_id'].values
#submission.drop('ATM_ID', inplace=True, axis=1)

y = model.predict(x, num_iteration=model.best_iteration)
y = [1 if i >=np.mean(y) else 0 for i in y]
output = pd.DataFrame({'ATM_ID': ids, 'PREDICT': y})
output.to_csv("out.csv", index=False)

In [131]:
output

,ATM_ID,PREDICT
0,0,1
1,1,0
2,2,0
3,3,0
4,4,1
5,5,1
6,6,1
7,7,1
8,8,1
9,9,0


In [132]:
output.shape

(351, 2)